# Generating samples (string patterns of length 32)

In this notebook we generate samples in the proper format we shall use to train. 

In [1]:
%%time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
import csv
import requests
from tqdm import tqdm
from collections import OrderedDict, defaultdict, Counter
import seaborn as sns
import json
from sklearn.decomposition import PCA
#from joblib import dump, load
import joblib

CPU times: user 582 ms, sys: 56.1 ms, total: 638 ms
Wall time: 636 ms


## Loading data

In [2]:
#parent_dir = "/content/genetic_engineering_attribution"
parent_dir = "/home/rio/data_sets/genetic_engineering_attribution"

#### Data

In [3]:
%%time
### pca directory
pca_dir = os.path.join(parent_dir,"pca")

### pca engineered data sets
pca_engineered_datasets_dir = os.path.join(parent_dir,"pca_engineered_datasets")

### train/val/test directories
full_train_dir = os.path.join(parent_dir,"full_train")
train_dir = os.path.join(parent_dir,"train") 
val_dir = os.path.join(parent_dir,"val")
test_dir = os.path.join(parent_dir,"test")

### Paths to csvs
full_train_path = os.path.join(full_train_dir,"full_train.csv")
train_path = os.path.join(train_dir,"train.csv")
val_path = os.path.join(val_dir,"val.csv")
test_path = os.path.join(test_dir,"test.csv")

### Dataframes
df_full_train = pd.read_csv(full_train_path,index_col=0)
df_train = pd.read_csv(train_path,index_col=0)
df_val = pd.read_csv(val_path,index_col=0)
df_test = pd.read_csv(test_path,index_col=0)

### Printing shapes:
print(f"Shape of df_full_train: {df_full_train.shape}")
print(f"Shape of df_train: {df_train.shape}")
print(f"Shape of df_val: {df_val.shape}")
print(f"Shape of df_test: {df_test.shape}")

Shape of df_full_train: (63017, 43)
Shape of df_train: (50413, 43)
Shape of df_val: (12604, 43)
Shape of df_test: (18816, 43)
CPU times: user 3.7 s, sys: 332 ms, total: 4.03 s
Wall time: 4.03 s


In [4]:
df_full_train.head()

,sequence_id,lab_id,sequence,seq_length,bacterial_resistance_ampicillin,bacterial_resistance_chloramphenicol,bacterial_resistance_kanamycin,bacterial_resistance_other,bacterial_resistance_spectinomycin,copy_number_high_copy,...,species_budding_yeast,species_fly,species_human,species_mouse,species_mustard_weed,species_nematode,species_other,species_rat,species_synthetic,species_zebrafish
0,9ZIMC,RYUA3GVO,CATGCATTAGTTATTAATAGTAATCAATTACGGGGTCATTAGTTCA...,7151,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5SAQC,RYUA3GVO,GCTGGATGGTTTGGGACATGTGCAGCCCCGTCTCTGTATGGAGTGA...,456,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,E7QRO,RYUA3GVO,NNCCGGGCTGTAGCTACACAGGGCGGAGATGAGAGCCCTACGAAAG...,1450,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,CT5FP,RYUA3GVO,GCGGAGATGAAGAGCCCTACGAAAGCTGAGCCTGCGACTCCCGCAG...,914,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7PTD8,RYUA3GVO,CGCGCATTACTTCACATGGTCCTCAAGGGTAACATGAAAGTGATCC...,1350,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


#### String patterns

In [5]:
%%time
### directory of string patterns
string_patterns_dir = os.path.join(parent_dir,"string_patterns")

### string patterns jsons
string_patterns_16 = os.path.join(string_patterns_dir,"string_patterns_16.json")

### choose string pattern
string_patterns_file = string_patterns_16
### loading string pattern
with open(string_patterns_file) as json_file:
    string_patterns = json.load(json_file)
print("Length of string_patterns_32: ", len(string_patterns))


Length of string_patterns_32:  3486651
CPU times: user 1.77 s, sys: 277 ms, total: 2.05 s
Wall time: 2.04 s


In [6]:
string_patterns

{'CATGCATTAGTTATTA': 180,
 'ATAGTAATCAATTACG': 448,
 'GGGTCATTAGTTCATA': 446,
 'GCCCATATATGGAGTT': 436,
 'CCGCGTTACATAACTT': 437,
 'ACGGTAAATGGCCCGC': 1134,
 'CTGGCTGACCGCCCAA': 470,
 'CGACCCCCGCCCATTG': 464,
 'ACGTCAATAATGACGT': 479,
 'ATGTTCCCATAGTAAC': 480,
 'GCCAATAGGGACTTTC': 487,
 'CATTGACGTCAATGGG': 2492,
 'TGGAGTATTTACGGTA': 469,
 'AACTGCCCACTTGGCA': 487,
 'GTACATCAAGTGTATC': 486,
 'ATATGCCAAGTACGCC': 527,
 'CCCTATTGACGTCAAT': 589,
 'GACGGTAAATGGCCCG': 575,
 'CCTGGCATTATGCCCA': 573,
 'GTACATGACCTTATGG': 533,
 'GACTTTCCTACTTGGC': 582,
 'AGTACATCTACGTATT': 576,
 'AGTCATCGCTATTACC': 576,
 'ATGGTGATGCGGTTTT': 463,
 'GGCAGTACATCAATGG': 449,
 'GCGTGGATAGCGGTTT': 476,
 'GACTCACGGGGATTTC': 466,
 'CAAGTCTCCACCCCAT': 464,
 'TGACGTCAATGGGAGT': 466,
 'TTGTTTTGGCACCAAA': 459,
 'ATCAACGGGACTTTCC': 461,
 'AAAATGTCGTAACAAC': 414,
 'TCCGCCCCATTGACGC': 416,
 'AAATGGGCGGTAGGCG': 483,
 'TGTACGGTGGGAGGTC': 469,
 'TATATAAGCAGAGCTG': 266,
 'GTTTAGTGAACCGTCA': 416,
 'GATCCGCTAGCGCTAC': 167,
 'CGGTCGCC

#### Fitted pca

In [7]:
%%time
###  directory with fitted pcas
pca_dir = os.path.join(parent_dir,"pca")

### pca_32_95comp
pca_16_48comp = os.path.join(pca_dir,"pca_16_48comp.joblib")

### loading
pca_16 = joblib.load(pca_16_48comp)

CPU times: user 3.08 ms, sys: 0 ns, total: 3.08 ms
Wall time: 2.13 ms


## Generating snippets

We now code a function to generate snippets. The snippets are sequences of uniform length and are generated from the original sequences.

In [8]:
def sample_window(array_of_lengths, l):
    #print("array_of_lengths: ", array_of_lengths)
    sampled_lower_bounds = [np.random.randint(0,(length - l)) for length in array_of_lengths]
    sampled_upper_bounds = [lb + l for lb in sampled_lower_bounds]
    sampled_windows = np.array([sampled_lower_bounds, sampled_upper_bounds]).T
    return sampled_windows
    
def generate_snippets(df, l, min_seq_length=None, size=None, replace = True, p="seq_length"):
    if min_seq_length is None:
        min_seq_length = l+1
    df_sampled = df[df.seq_length>=min_seq_length].copy()
    ### selecting indices
    #a, p, replace = df.index.values, None, False
    a = df_sampled.index.values
    if size is None:
        size = len(a)
    else:
        if p == "seq_length":
            p = df_sampled.seq_length.values/np.sum(df_sampled.seq_length.values)
            replace = True
    #print("a: ", a, type(a))
    #print("size: ", size, type(size))
    #print("replace: ", replace, type(replace))
    #print("p: ", p, type(p))
    sampled_ixs = np.random.choice(a, size, replace, p)
    #print("sampled_ixs: ", sampled_ixs)
    df_sampled = df_sampled.loc[sampled_ixs,:] 
    array_of_lengths = df_sampled.seq_length.values
    #if np.any(array_of_lengths<min_seq_length):
    #    print("Found one!")
    sampled_windows = sample_window(array_of_lengths, l) 
    df_sampled.loc[:,"sequence"] = [seq[l:u] for seq,(l,u) in zip(df_sampled.sequence.values, sampled_windows)]
    df_sampled.loc[:,"seq_length"] = [len(seq) for seq in df_sampled.sequence.values]
    #return df_sampled,sampled_windows, array_of_lengths, sampled_ixs
    return df_sampled

def generate_lab_snippets(df, l, min_seq_length=None, size=None, replace = True, p="seq_length",test=False):
    train = not test
    concat = []
    if train:
        pbar = tqdm(np.sort(df.lab_id.unique()))
        for lab in pbar:
            pbar.set_description(f"Processing lab {lab}")
            df_lab = df.loc[df.lab_id == lab,:]
            df_snippets = generate_snippets(df_lab, l, min_seq_length, size, replace, p)
            concat.append(df_snippets)
    else:
        pbar = tqdm(np.sort(df.sequence_id.unique()))
        for seq in pbar:
            pbar.set_description(f"Processing sequence {seq}")
            df_seq = df.loc[df.sequence_id == seq,:]
            df_snippets = generate_snippets(df_seq, l, min_seq_length, size, replace, p)
            concat.append(df_snippets)       
    df_samples = pd.concat(concat)
    return df_samples
    

#### Generating full training snippets 

In [9]:
%%time
np.random.seed(93652)
l= 16
min_seq_length = None
size = 1500
replace = True
p="seq_length"
test=False
full_train_snippets = generate_lab_snippets(df_full_train,l,min_seq_length,size,replace,p,test)
print("Shape of full train_snippets: ", full_train_snippets.shape)
full_train_snippets.sample(20)

Processing lab ZZJVE4HO: 100%|██████████| 1314/1314 [00:15<00:00, 84.44it/s]


Shape of full train_snippets:  (1971000, 43)
CPU times: user 15.9 s, sys: 692 ms, total: 16.6 s
Wall time: 16 s


,sequence_id,lab_id,sequence,seq_length,bacterial_resistance_ampicillin,bacterial_resistance_chloramphenicol,bacterial_resistance_kanamycin,bacterial_resistance_other,bacterial_resistance_spectinomycin,copy_number_high_copy,...,species_budding_yeast,species_fly,species_human,species_mouse,species_mustard_weed,species_nematode,species_other,species_rat,species_synthetic,species_zebrafish
13753,3ZQY9,7ANCD9AK,TGGTGGAAGTCGGCCA,16,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43752,GWT3Q,9PWYZMNS,GAGAATATGTTTTTCG,16,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
39280,ULXPM,Y81SHRRC,TATCCATTTTCGGATC,16,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
60991,65U1S,C1BIUBL5,ACTATTGCACGTTTCA,16,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5333,EXZHK,L78GOBQS,GCAAGCGGCCGATGCC,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60693,BR4SY,H5Y73UHQ,GCGCAGAAAAAAAGGA,16,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
32730,RJPID,LUHRMKEB,CCAACAGTACCGGAAT,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
25797,HS943,K212MH7P,AGCGGGAGTCTGCACA,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
35196,FFG0U,SR345GAS,TTGGCCAGAACGTGCT,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9188,49R6R,THD393NW,AGAATTTGTCCACTAC,16,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
full_train_snippets.seq_length.unique()

array([16])

#### Generating validation snippets 

In [17]:
%%time
#np.random.seed(8135)
#l=16
#min_seq_length = None
#size = 1000
#replace = True
#p="seq_length"
#test = False
#val_snippets = generate_lab_snippets(df_val,l,min_seq_length,size,replace,p,test)
#print("Shape of val_snippets: ", val_snippets.shape)
#val_snippets.sample(10)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs


In [18]:
#val_snippets.seq_length.unique()

#### Generating test snippets

To generate test snippets, we first "augment" the sequences that are not long enough.

In [13]:
def augment_sequence_by_repetition(df,min_seq_length):
    df = df.copy()
    locs = df_test.seq_length<min_seq_length
    lengths = np.array([len(seq) for seq in df.loc[locs,"sequence"]])
    factors  = np.ceil(min_seq_length/lengths).astype(int)
    df.loc[locs,"sequence"] = df.loc[locs,"sequence"]*factors
    ### updating lengths of sequences
    df.loc[locs,"seq_length"] = [len(seq) for seq in df.loc[locs,"sequence"]]
    return df

In [14]:
%%time
min_seq_length = 16
df_test_augmented = augment_sequence_by_repetition(df_test,min_seq_length)

CPU times: user 9.93 ms, sys: 0 ns, total: 9.93 ms
Wall time: 8.79 ms


#### Checking augmentation

In [15]:
dci = {}
for ix, (seq, seq_len) in enumerate(zip(df_test_augmented.loc[:,"sequence"].values, df_test_augmented.loc[:,"seq_length"].values)):
    if len(seq)<16 or len(seq)!=seq_len:
        dci[ix] = (seq,seq_len)
dci

{}

In [16]:
%%time
np.random.seed(67438)
l=16
min_seq_length = None
size = 100
replace = True
p="seq_length"
test = True
test_snippets = generate_lab_snippets(df_test_augmented,l,min_seq_length,size,replace,p,test)
print("Shape of test_snippets: ", test_snippets.shape)
test_snippets.sample(10)

Processing sequence ZZZ9J: 100%|██████████| 18816/18816 [01:08<00:00, 273.10it/s]


Shape of test_snippets:  (1881600, 43)
CPU times: user 1min 17s, sys: 3.68 s, total: 1min 21s
Wall time: 1min 15s


,sequence_id,lab_id,sequence,seq_length,bacterial_resistance_ampicillin,bacterial_resistance_chloramphenicol,bacterial_resistance_kanamycin,bacterial_resistance_other,bacterial_resistance_spectinomycin,copy_number_high_copy,...,species_budding_yeast,species_fly,species_human,species_mouse,species_mustard_weed,species_nematode,species_other,species_rat,species_synthetic,species_zebrafish
7703,AEK3O,NaN,GCGCCACATAGCAGAA,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4535,TBYDI,NaN,AGCCCCTCCCACGACG,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5402,90613,NaN,ATTTGAGGCGCTAAAT,16,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4566,1OF6U,NaN,AAGCCACGACGGAGGA,16,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
13143,G7241,NaN,CTTCTTGAGATCCTTT,16,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18047,FQX2X,NaN,GGCCAAGCCTAGGCCC,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10292,FHUR1,NaN,GTTAATTGCGCGCTTG,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1735,BR734,NaN,CAGTGGAGGCTGAGAT,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1993,RUX5C,NaN,CAGCCCCGACACCCGC,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2147,WZYKR,NaN,TGGAGCGAACGACCTA,16,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [19]:
test_snippets.seq_length.unique()

array([16])

## Mapping string patterns to vectors

In [20]:
def str2vec(sequence,keywords={"A":[1,0,0,0,0], "T":[0,1,0,0,0], "G":[0,0,1,0,0], "C":[0,0,0,1,0], "N":[0,0,0,0,1]}):
    vec = []
    for s in sequence:
        vec.extend(keywords[s])
    return np.array(vec)

def str2mat(sequences,keywords={"A":[1,0,0,0,0], "T":[0,1,0,0,0], "G":[0,0,1,0,0], "C":[0,0,0,1,0], "N":[0,0,0,0,1]}):
    mat = np.array([str2vec(seq) for seq in tqdm(sequences)])
    return mat

## Generating PCA engineered data sets

In [22]:
full_train_snippets.columns.get_loc("sequence")

2

In [23]:
def add_pca_features(df,pca):
    mat = str2mat(df.sequence.values)
    mat = pca.transform(mat)
    meta_cols = list(df.columns[:df.columns.get_loc("bacterial_resistance_ampicillin")])
    data_cols = list(df.columns[df.columns.get_loc("bacterial_resistance_ampicillin"):])
    pca_cols = [f"pca_{ix}".zfill(1) if ix <10 else f"pca_{ix}" for ix in range(mat.shape[1])]
    cols = meta_cols + pca_cols + data_cols 
    df_meta = df.loc[:,meta_cols]
    df_pca = pd.DataFrame(mat,columns=pca_cols,index=df.index)
    df_data = df.loc[:,data_cols]
    df = pd.concat([df_meta,df_pca,df_data],axis=1)
    return df

#### Full training set

In [24]:
%%time
df_full_train_pca = add_pca_features(full_train_snippets,pca_16)
print(df_full_train_pca.shape)

100%|██████████| 1971000/1971000 [00:12<00:00, 160872.78it/s]


(1971000, 91)
CPU times: user 15.1 s, sys: 661 ms, total: 15.8 s
Wall time: 14.8 s


#### Validation set 

In [25]:
%%time
#df_val_pca = add_pca_features(val_snippets,pca_32)
#print(df_val_pca.shape)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs


#### Test set

In [26]:
%%time
df_test_pca = add_pca_features(test_snippets,pca_16)
print(df_test_pca.shape)

100%|██████████| 1881600/1881600 [00:11<00:00, 160327.43it/s]


(1881600, 91)
CPU times: user 14.9 s, sys: 877 ms, total: 15.8 s
Wall time: 14.9 s


## Saving datasets with PCA engineered features

In [31]:
def save_datasets(savedir,dfs_dict,overwrite=True):
    pbar = tqdm(dfs_dict.items(),total=len(dfs_dict))
    if not os.path.isdir(savedir):
        print(f"Creating directory {savedir}")
        os.makedirs(savedir)
    elif overwrite:
        print(f"Directory {savedir} already exists. Overwriting.")
        shutil.rmtree(savedir)
        os.makedirs(savedir)
    else:
        print(f"Directory {savedir} already exists. Skipping.")
    for f, df in pbar:
        savepath = os.path.join(savedir,f+".csv")
        df.to_csv(savepath,index=True)
        

#### Saving 

In [32]:
%%time
savedir = os.path.join(pca_engineered_datasets_dir,"pca_16_48comp","train_val_test")
#dfs_dict = {"train": df_train_pca, "val": df_val_pca, "test": df_test_pca}
dfs_dict = {"full_train": df_full_train_pca, "test": df_test_pca}
overwrite=True
save_datasets(savedir,dfs_dict,overwrite)


  0%|          | 0/2 [00:00<?, ?it/s]

Creating directory /home/rio/data_sets/genetic_engineering_attribution/pca_engineered_datasets/pca_16_48comp/train_val_test



100%|██████████| 2/2 [04:24<00:00, 132.45s/it]

CPU times: user 4min 21s, sys: 2.63 s, total: 4min 23s
Wall time: 4min 24s
